Learning Pytorch

Quick Start section

In [39]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Dataset store the sample and the label
# DataLoader waro around Dataset

print(f"is CUDA available for torch? {torch.cuda.is_available()}")
print(torch.__version__)

is CUDA available for torch? True
2.3.1+cu118


In [40]:
# Download data from open Dataset
# FashionMNIST dataset

training_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())

test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())


In [41]:
batch_size = 64

# data loader
# support automatic batching, sampling, shuffling and multiprocess data loading
# batch size = # of sample that is going to propagate through the network

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of Y: {y.shape}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64])


In [42]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [43]:
# To define a neural net

class NueralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [44]:
model = NueralNetwork().to(device)
print(model)

NueralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [45]:
lose_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [46]:
def train(dataloader, model, loss_function, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_function(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d} / {size:>5d}]")
    

In [47]:
def test(dataloader, model, lose_function):
    size = len(dataloader.dataset)
    num_batchs = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += lose_function(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batchs
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    

In [48]:
epochs = 5
model.to('cuda')

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, lose_function, optimizer)
    test(test_dataloader, model, lose_function)
print("Done!")

Epoch 1
-------------------------------
loss: 2.290290 [   64 / 60000]
loss: 2.264224 [ 6464 / 60000]
loss: 2.224688 [12864 / 60000]
loss: 2.229846 [19264 / 60000]
loss: 2.180174 [25664 / 60000]
loss: 2.117064 [32064 / 60000]
loss: 2.138170 [38464 / 60000]
loss: 2.062141 [44864 / 60000]
loss: 2.058167 [51264 / 60000]
loss: 1.995642 [57664 / 60000]
Test Error: 
 Accuracy: 54.7%, Avg loss: 1.990338 

Epoch 2
-------------------------------
loss: 2.003455 [   64 / 60000]
loss: 1.982643 [ 6464 / 60000]
loss: 1.871408 [12864 / 60000]
loss: 1.907773 [19264 / 60000]
loss: 1.800393 [25664 / 60000]
loss: 1.726261 [32064 / 60000]
loss: 1.739608 [38464 / 60000]
loss: 1.629402 [44864 / 60000]
loss: 1.639040 [51264 / 60000]
loss: 1.531567 [57664 / 60000]
Test Error: 
 Accuracy: 63.2%, Avg loss: 1.554538 

Epoch 3
-------------------------------
loss: 1.606279 [   64 / 60000]
loss: 1.575458 [ 6464 / 60000]
loss: 1.419807 [12864 / 60000]
loss: 1.491731 [19264 / 60000]
loss: 1.363459 [25664 / 60000]
l

In [49]:
# Saving the model

torch.save(model.state_dict(), "model.pth")
print("Save Pytorch Model State to model.pth")

Save Pytorch Model State to model.pth


In [53]:
# model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [55]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Ankle boot", Actual: "Ankle boot"
